In [55]:
import pandas as pd
import numpy as np
df=pd.read_csv('data/train.csv')

Model Taining

In [56]:
df.drop(columns='id',axis=1,inplace=True)

In [57]:
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [58]:
X=df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [59]:
categorical_columns=X.select_dtypes(include ='object').columns
numereical_columns=X.select_dtypes(exclude='object').columns

In [60]:

df['clarity'].unique()

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [61]:
cut_categories=['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
color_categories=['F', 'J', 'G', 'E', 'D', 'H', 'I']
clarity_categories=['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']

In [62]:
from sklearn.impute import SimpleImputer #Handling Missing values
from sklearn.preprocessing import StandardScaler #handle Feature Scaling
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding

##pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



In [63]:
##Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

## Categorical Pipeline
cat_Pipeline= Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,numereical_columns),
        ('cat_Pipeline',cat_Pipeline,categorical_columns)
    ]
)

In [64]:
##Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)

In [65]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [66]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_Pipeline__cut,cat_Pipeline__color,cat_Pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.652424,0.058753,1.605394
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-0.381506,1.123070,-0.079246
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-1.415437,1.123070,-1.202339
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.652424,-1.537723,2.166941
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-0.381506,1.655229,0.482301


In [68]:
##Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [69]:
regeression = LinearRegression()
regeression.fit(X_train,y_train)

LinearRegression()

In [78]:
import numpy as np
def evalution_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [82]:
##Train multiple model
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}

model_list=[]
r2_list=[]
trained_model_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #making a  prediction
    y_pred=model.predict(X_test)

    mae,rmse,r2_square=evalution_model(y_test,y_pred)

    print(list(models.values())[i])
    model_list.append(list(models.values())[i])
    # trained_model_list.append(model.values()[i])
    
    print('Model Training Performance')
    print('RMSE:',rmse*100)
    print('MAE',mae)
    print("r2 score",r2_square*100)
    r2_list.append(r2_square*100)
    print('='*35)
    print('\n')

LinearRegression()
Model Training Performance
RMSE: 119682.48815972514
MAE 723.6034569047349
r2 score 91.13560571385317


Lasso()
Model Training Performance
RMSE: 119679.36860863608
MAE 725.055055982241
r2 score 91.13606781270963


Ridge()
Model Training Performance
RMSE: 119683.07856345207
MAE 723.6370749375782
r2 score 91.1355182560402


ElasticNet()
Model Training Performance
RMSE: 163253.94144265962
MAE 1135.0835947511694
r2 score 83.5064099195437




In [83]:
model_list

[LinearRegression(), Lasso(), Ridge(), ElasticNet()]